# VGG 16


## Implementation of VGG-16 in Keras

In [ ]:
import keras,os
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import matplotlib.pyplot as plt
import glob
import tensorflow as tf
import pandas as pd
import cv2

In [ ]:
from tensorflow.keras.datasets import cifar10

# importing the cifar-10 datasets. This dataset contains 10 classes of objects.
# It contains 5000 training examples and 1000 test images for each class

(x_train,y_train),(x_test, y_test) = cifar10.load_data()

In [ ]:
x_train,y_train = tf.convert_to_tensor(x_train, dtype = tf.float32),tf.convert_to_tensor(y_train, dtype = tf.float32)
x_test, y_test = tf.convert_to_tensor(x_test, dtype = tf.float32), tf.convert_to_tensor(y_test, dtype = tf.float32)

In [ ]:
# Let us create the model now 

model = Sequential()
model.add(tf.keras.layers.Resizing(224,224))
model.add(Conv2D(input_shape=(224,224,3),filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

model.add(Flatten())
model.add(Dense(units=4096,activation="relu"))
model.add(Dense(units=4096,activation="relu"))
model.add(Dense(units=10, activation="softmax"))

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics=['accuracy'])

In [ ]:
history = model.fit(x_train, y_train, batch_size = 100, epochs = 10)

In [ ]:
model.summary()

In [ ]:
train_loss = history.history['loss']
train_accuracy= history.history['accuracy']

In [ ]:
results = model.evaluate(x_test, y_test, batch_size = 32)
print("test loss, test acc:", results)

In [ ]:
plt.figure()
plt.title("Error of the model vs the number of iterations")
plt.plot(range(1,21), train_loss, label = "Training Loss")
plt.xlabel("Number of Iterations")
plt.ylabel("Error of the model")
plt.legend()
plt.show()

plt.figure()
plt.title("Accuracy of the model vs the number of iterations")
plt.plot(range(1,21), np.array(train_accuracy)*100, label = 'Training Accuracy')
plt.xlabel("Number of Iterations")
plt.ylabel("Accuracy of the model")
plt.legend()
plt.show()

## Implementation of VGG16 in Pytorch

In [ ]:
import numpy as np
import torch
import torch.nn as nn
from torchvision import datasets
from torchvision import transforms
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import DataLoader, Dataset


# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
stats = ((0.5074,0.4867,0.4411),(0.2011,0.1987,0.2025))
transform = transforms.Compose([transforms.Resize((224, 224)),transforms.ToTensor(),transforms.Normalize(*stats)])

In [ ]:
# Let us use Cifar-100 datasets for model in pytorch

train_data = datasets.CIFAR10(download=True,root="./data",transform = transform)
test_data = datasets.CIFAR10(root="./data",train=False, transform = transform)

In [ ]:
train_loader = DataLoader(train_data, batch_size = 32, shuffle = True)
test_loader = DataLoader(test_data, batch_size = 32, shuffle = True)

In [ ]:
class VGG16(nn.Module):
    def __init__(self):
        super(VGG16, self).__init__()
        
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU())
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(), 
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        
        self.layer3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU())
        
        self.layer4 = nn.Sequential(
            nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        
        self.layer5 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU())
        
        self.layer6 = nn.Sequential(
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU())
        
        self.layer7 = nn.Sequential(
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        
        self.layer8 = nn.Sequential(
            nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU())
        
        self.layer9 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU())
        
        self.layer10 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        
        self.layer11 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU())
        
        self.layer12 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU())
        
        self.layer13 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        
        self.fc = nn.Sequential(
            nn.Linear(7*7*512, 4096),
            nn.ReLU())
        
        self.fc1 = nn.Sequential(
            nn.Linear(4096, 4096),
            nn.ReLU())
        
        self.fc2= nn.Sequential(
            nn.Linear(4096, 10))
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.layer5(out)
        out = self.layer6(out)
        out = self.layer7(out)
        out = self.layer8(out)
        out = self.layer9(out)
        out = self.layer10(out)
        out = self.layer11(out)
        out = self.layer12(out)
        out = self.layer13(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        out = self.fc1(out)
        out = self.fc2(out)
        return out

In [ ]:
epochs = 10
model = VGG16().to(device)
optim = torch.optim.SGD(model.parameters(), lr=0.01,momentum=0.9)
criterion = nn.CrossEntropyLoss()
training_loss, testing_loss = [],[]
training_accuracy, testing_accuracy = [],[]

In [ ]:
def multi_acc(y_pred, target):
    y_pred_softmax = torch.log_softmax(y_pred, dim = 1)
    max_val, y_tags = torch.max(y_pred_softmax, dim = 1)
    
    correct_pred = (y_tags == target).float()
    acc = correct_pred.sum() / len(correct_pred)
    
    return acc

In [ ]:
for epoch in range(epochs):
    print("Epoch :",epoch+1)
    batch_train_loss, batch_test_loss = [],[]
    batch_train_accuracy, batch_test_accuracy = [],[]
    
    for cnt,(x_val, y_val) in enumerate(train_loader) :
        x_val, y_val = x_val.to(device), y_val.to(device)
        optim.zero_grad()
        output = model(x_val)
        train_loss = criterion(output, y_val)
        train_loss.backward()
        optim.step()
        with torch.no_grad():
             batch_train_loss.append(train_loss.cpu().detach().numpy())
             train_accuracy = multi_acc(output, y_val)
             batch_train_accuracy.append(train_accuracy.cpu().detach().numpy())
             print("Batch number {}/1563 Training Accuracy {}, Training Loss {}".format(cnt+1,train_accuracy, train_loss))
 
    for cnt,(x, y) in enumerate(test_loader) :
        x, y = x.to(device), y.to(device)
        out = model(x)
        test_loss = criterion(out, y)
        batch_test_loss.append(test_loss.cpu().detach().numpy())
        test_accuracy = multi_acc(out, y)
        batch_test_accuracy.append(test_accuracy.cpu().detach().numpy())
        print("Batch number {}/313 Testing Accuracy {}, Testing Loss {}".format(cnt+1,test_accuracy, test_loss))
        
    training_loss.append(np.array(batch_train_loss).mean())
    training_accuracy.append(np.array(batch_train_accuracy).mean())
    testing_loss.append(np.array(batch_test_loss).mean())
    testing_accuracy.append(np.array(batch_test_accuracy).mean())

In [ ]:
plt.figure()
plt.title("Evolution of loss function")
plt.plot(range(epochs),training_loss, label = "Training Loss")
plt.plot(range(epochs), testing_loss, label = "Testing loss")
plt.xlabel("Number of iterations")
plt.ylabel("Loss in the model")
plt.legend()
plt.show()

plt.figure()
plt.title("Evolution of Accuracy")
plt.plot(range(epochs), training_accuracy, label = "Training Accuracy")
plt.plot(range(epochs), testing_accuracy, label = "Testing Accuracy")
plt.xlabel("Number of iterations")
plt.ylabel("Accuracy of the model")
plt.legend()
plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix,classification_report
model.to(torch.device('cpu'))
y_pred = model(x_test)
y_pred_softmax = torch.log_softmax(y_pred, dim = 1)
max_val, y_tags = torch.max(y_pred_softmax, dim = 1)

print("Confusion Matrix for the model is")
print(confusion_matrix(y_test, y_tags))

print()